In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense
from tensorflow.keras.optimizers import Adam

## Relu Funtion Code 

# Loading and preprocessing of CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Function to build deeper CNN model with ReLU function
def build_deeper_relu_model(config):
    model = Sequential()
    for i in range(config['layers']):
        model.add(Conv2D(config['filters'][i], (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3) if i == 0 else None))
        model.add(BatchNormalization())
        if i < config['layers'] - 1:  
            model.add(MaxPooling2D((2, 2)))
        else:
            model.add(tf.keras.layers.GlobalAveragePooling2D())  
        if 'dropout' in config:
            model.add(Dropout(config['dropout']))
    model.add(Dense(config['dense_units'], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=config['lr']), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Plotting accuracy and loss curves
def plot_curves(history, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    # Accuracy curves
    ax1.plot(history['accuracy'], label='Training Accuracy')
    ax1.plot(history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title(f'{title} - Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    # Loss curves
    ax2.plot(history['loss'], label='Training Loss')
    ax2.plot(history['val_loss'], label='Validation Loss')
    ax2.set_title(f'{title} - Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend()

    plt.show()

#  predictions vs actual labels
def show_predictions(model, x_test, y_test, num_images=10):
    predictions = model.predict(x_test[:num_images])
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test[:num_images], axis=1)

    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    for i in range(num_images):
        axes[i].imshow(x_test[i])
        axes[i].axis('off')
        axes[i].set_title(f"True: {true_labels[i]}\nPred: {predicted_labels[i]}")
    plt.show()

# Model configurations
configs = {
    'shallow': {'layers': 2, 'filters': [32, 64], 'dense_units': 128, 'dropout': 0.2, 'lr': 0.001, 'epochs': 10},
    'medium':  {'layers': 3, 'filters': [32, 64, 128], 'dense_units': 256, 'dropout': 0.3, 'lr': 0.001, 'epochs': 20},
    'deep':    {'layers': 4, 'filters': [32, 64, 128, 256], 'dense_units': 512, 'dropout': 0.4, 'lr': 0.0005, 'epochs': 30},
    'deeper':  {'layers': 6, 'filters': [32, 64, 128, 256, 512, 512], 'dense_units': 1024, 'dropout': 0.5, 'lr': 0.0005, 'epochs': 30},
}

# Train, evaluate, and plot for ReLU
history_dict = {}
results = {}
models = {}

for name, config in configs.items():
    print(f"Training {name} configuration with ReLU activation...")
    model = build_deeper_relu_model(config)
    history = model.fit(x_train, y_train, epochs=config['epochs'], batch_size=128, validation_split=0.2, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

    history_dict[name] = history.history
    results[name] = {'test_accuracy': test_acc, 'test_loss': test_loss}
    models[name] = model 

# Results
print("\nFinal Results:")
for config_name, metrics in results.items():
    print(f"{config_name} - Test Accuracy: {metrics['test_accuracy']:.4f}, Test Loss: {metrics['test_loss']:.4f}")

# Plot accuracy/loss curves for deep configuration
print("\nReLU Accuracy and Loss Curves for Deep Configuration")
plot_curves(history_dict['deep'], "ReLU Deep Configuration")

# Show predictions for deep configuration
print("\nActual vs Predicted Outputs for ReLU (Deep Configuration)")
show_predictions(models['deep'], x_test, y_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np


## Leaky ReLU function Code


# Loading and preprocessing CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Function to build deeper CNN model with Leaky ReLU activation
def build_deeper_leaky_relu_model(config):
    model = Sequential()
    for i in range(config['layers']):
        model.add(Conv2D(config['filters'][i], (3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.01),
                         padding='same', input_shape=(32, 32, 3) if i == 0 else None))
        model.add(BatchNormalization())
        if i < config['layers'] - 1:  
            model.add(MaxPooling2D((2, 2)))
        else:
            model.add(tf.keras.layers.GlobalAveragePooling2D())  
        if 'dropout' in config:
            model.add(Dropout(config['dropout']))
    model.add(Dense(config['dense_units'], activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=config['lr']), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# configurations
configs = {
    'shallow': {'layers': 2, 'filters': [32, 64], 'dense_units': 128, 'dropout': 0.2, 'lr': 0.001, 'epochs': 10},
    'medium':  {'layers': 3, 'filters': [32, 64, 128], 'dense_units': 256, 'dropout': 0.3, 'lr': 0.001, 'epochs': 20},
    'deep':    {'layers': 4, 'filters': [32, 64, 128, 256], 'dense_units': 512, 'dropout': 0.4, 'lr': 0.0005, 'epochs': 30},
    'deeper':  {'layers': 6, 'filters': [32, 64, 128, 256, 512, 512], 'dense_units': 1024, 'dropout': 0.5, 'lr': 0.0005, 'epochs': 30},
}

# Train and evaluate models for each configuration
history_dict = {}
results = {}
models = {}

for name, config in configs.items():
    print(f"Training {name} configuration with Leaky ReLU activation...")
    model = build_deeper_leaky_relu_model(config)
    history = model.fit(x_train, y_train, epochs=config['epochs'], batch_size=128, validation_split=0.2, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

    history_dict[name] = history.history
    results[name] = {'test_accuracy': test_acc, 'test_loss': test_loss}
    models[name] = model

#  Results
print("\nFinal Results:")
for config_name, metrics in results.items():
    print(f"{config_name} - Test Accuracy: {metrics['test_accuracy']:.4f}, Test Loss: {metrics['test_loss']:.4f}")

# Plot accuracy and loss curves
def plot_curves(history, title):
    epochs = range(1, len(history['accuracy']) + 1)
    plt.figure(figsize=(12, 5))

    # Accuracy Plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['accuracy'], label='Training Accuracy')
    plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss Plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['loss'], label='Training Loss')
    plt.plot(epochs, history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Function to display Actual vs Predicted Outputs
def show_actual_vs_predicted(model, x_test, y_test, class_names, num_images=10):
    y_pred = np.argmax(model.predict(x_test[:num_images]), axis=1)
    y_true = np.argmax(y_test[:num_images], axis=1)

    plt.figure(figsize=(12, 2))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(x_test[i])
        plt.axis('off')
        plt.title(f"True: {y_true[i]}\nPred: {y_pred[i]}")
    plt.show()

# Plot Accuracy/Loss Curves for Leaky ReLU - Deep Configuration
print("Leaky ReLU Accuracy and Loss Curves")
plot_curves(history_dict['deep'], "Leaky ReLU Deep Configuration")

# Show Actual vs Predicted Outputs for Leaky ReLU - Deep Configuration
print("\nActual vs Predicted Outputs for Leaky ReLU")
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
show_actual_vs_predicted(models['deep'], x_test, y_test, class_names)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import cifar10

## Swish Function Code

# Loading and preprocessing CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Function to build CNN model with Swish activation
def build_deeper_swish_model(config):
    model = Sequential()
    for i in range(config['layers']):
        model.add(Conv2D(config['filters'][i], (3, 3), activation=tf.keras.activations.swish, padding='same', input_shape=(32, 32, 3) if i == 0 else None))
        model.add(BatchNormalization())
        if i < config['layers'] - 1: 
            model.add(MaxPooling2D((2, 2)))
        else:
            model.add(tf.keras.layers.GlobalAveragePooling2D())  
        if 'dropout' in config:
            model.add(Dropout(config['dropout']))
    model.add(Dense(config['dense_units'], activation=tf.keras.activations.swish))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=config['lr']), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Configurations for Swish activation 
configs = {
    'shallow': {'layers': 2, 'filters': [32, 64], 'dense_units': 128, 'dropout': 0.2, 'lr': 0.001, 'epochs': 10},
    'medium':  {'layers': 3, 'filters': [32, 64, 128], 'dense_units': 256, 'dropout': 0.3, 'lr': 0.001, 'epochs': 20},
    'deep':    {'layers': 4, 'filters': [32, 64, 128, 256], 'dense_units': 512, 'dropout': 0.4, 'lr': 0.0005, 'epochs': 30},
    'deeper':  {'layers': 6, 'filters': [32, 64, 128, 256, 512, 512], 'dense_units': 1024, 'dropout': 0.5, 'lr': 0.0005, 'epochs': 30},
}

# Training and evaluate models for each configuration
swish_history_dict = {}
swish_results = {}

for name, config in configs.items():
    print(f"Training {name} configuration with Swish activation...")
    model = build_deeper_swish_model(config)
    history = model.fit(x_train, y_train, epochs=config['epochs'], batch_size=128, validation_split=0.2, verbose=2)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

    swish_history_dict[name] = history.history
    swish_results[name] = {'test_accuracy': test_acc, 'test_loss': test_loss}

# Results
print("\nFinal Results:")
for config_name, metrics in swish_results.items():
    print(f"{config_name} - Test Accuracy: {metrics['test_accuracy']:.4f}, Test Loss: {metrics['test_loss']:.4f}")

# Accuracy and Loss Curves
def plot_curves(history, title):
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    # Accuracy
    axs[0].plot(history['accuracy'], label='Training Accuracy')
    axs[0].plot(history['val_accuracy'], label='Validation Accuracy')
    axs[0].set_title(f'{title} - Accuracy')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend()

    # Loss
    axs[1].plot(history['loss'], label='Training Loss')
    axs[1].plot(history['val_loss'], label='Validation Loss')
    axs[1].set_title(f'{title} - Loss')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Loss')
    axs[1].legend()
    plt.show()

# Plot accuracy/loss curves for Swish
print("Swish Accuracy and Loss Curves")
plot_curves(swish_history_dict['deep'], "Swish Deep Configuration")

# Show Predicted vs Actual Outputs
def show_predictions(model, x_test, y_test, num_images=10):
    predictions = model.predict(x_test[:num_images])
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test[:num_images], axis=1)

    plt.figure(figsize=(15, 3))
    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(x_test[i])
        plt.title(f"True: {true_labels[i]}\nPred: {predicted_labels[i]}")
        plt.axis('off')
    plt.show()

# Display predicted vs actual outputs for the deep configuration
print("Swish Predicted vs Actual Outputs")
deep_swish_model = build_deeper_swish_model(configs['deep'])
deep_swish_model.fit(x_train, y_train, epochs=configs['deep']['epochs'], batch_size=128, validation_split=0.2, verbose=0)
show_predictions(deep_swish_model, x_test, y_test)